Import modules

In [10]:
#import modules
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import geocoder

Download contents of webpage

In [11]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text

Create Beautiful Soup object

In [12]:
soup = BeautifulSoup(data,"html5lib")

Create dataframe from beautiful soup object

In [13]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

Get Lat. and Long. of Neighbourhoods

In [15]:
postal_code = df['PostalCode']

latitude = []
longitude = []
n = 0

while n < len(postal_code):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code[n]))
    lat_lng_coords = g.latlng
    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])
    n = n + 1

df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


Define new dataframe that only contains the word Toronto in boroughs

In [17]:
df_torro = df[df['Borough'].str.contains("Toronto")]
df_torro.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
15,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
19,M4E,East Toronto,The Beaches,43.67709,-79.29547
20,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


Enter Foursquare Credentials

In [105]:
# Define Foursquare credentials
CLIENT_ID = 'XXXX'
CLIENT_SECRET = 'XXX'
VERSION = '20180605'
LIMIT = 100 

Create function to get data off foursquare

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run function to create new dataframe

In [30]:
torro_venues = getNearbyVenues(names=df_torro['Neighborhood'],
                                   latitudes=df_torro['Latitude'],
                                   longitudes=df_torro['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [49]:
torro_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


Run one hot encoding on dataframe

In [48]:
# one hot encoding
torro_onehot = pd.get_dummies(torro_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torro_onehot['Neighborhood'] = torro_venues['Neighborhood']

# move neighborhood column to the first column
hood = torro_onehot['Neighborhood']
torro_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
torro_onehot.insert(0, 'Neighborhood', hood)

torro_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighbourhood and mean of category

In [78]:
torro_grouped = torro_onehot.groupby('Neighborhood').mean().reset_index()
torro_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0,0.0,0.015625,0.0,0.015625,0.000000,0.000000,0.000000,...,0.0,0.0,0.015625,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.015625
1,"Brockton, Parkdale Village, Exhibition Place",0.011628,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.023256,0.000000,...,0.0,0.0,0.011628,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.023256
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.013158,...,0.0,0.0,0.000000,0.013158,0.000000,0.000000,0.000000,0.0,0.0,0.013158
3,Central Bay Street,0.000000,0.0,0.0,0.000000,0.0,0.017544,0.017544,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.017544,0.017544,0.017544,0.0,0.0,0.000000
4,Christie,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000


create function to sort venues in descending order and create dataframe with top ten venues by neighbourhood

In [88]:
import numpy as np

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torro_grouped['Neighborhood']

for ind in np.arange(torro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Breakfast Spot,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Beer Bar
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Café,Coffee Shop,Restaurant,Sandwich Place,Nightclub,Bakery,Gift Shop,Yoga Studio,Lounge
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Sandwich Place,French Restaurant,Park,Bar,Restaurant,Speakeasy,Bakery
3,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Restaurant,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Sandwich Place,Café,Bookstore
4,Christie,Café,Grocery Store,Italian Restaurant,Playground,Baby Store,Athletics & Sports,Coffee Shop,Candy Store,Farmers Market,Farm


Run k-means to cluster neighbourhoods into 5 clusters

In [89]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

torro_grouped_clustering = torro_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torro_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [90]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torro_merged = df_torro

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
torro_merged = torro_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torro_merged = torro_merged.reset_index(drop=True)
torro_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Bakery,Distribution Center,Electronics Store,Event Space,Food Truck,Gym / Fitness Center
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Bubble Tea Shop,Theater
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Coffee Shop,Cosmetics Shop,Hotel,Italian Restaurant,Cocktail Bar,Clothing Store,Café,Park,Seafood Restaurant,Gym
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,0,Health Food Store,Pub,Trail,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Breakfast Spot,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Beer Bar


Visualise clusters

In [98]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="torro_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torro_merged['Latitude'], torro_merged['Longitude'], torro_merged['Neighborhood'], torro_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examine clusters

In [99]:
torro_merged.loc[torro_merged['Cluster Labels'] == 0, torro_merged.columns[[1] + list(range(5, torro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Yoga Studio,Thai Restaurant,Bakery,Distribution Center,Electronics Store,Event Space,Food Truck,Gym / Fitness Center
1,Downtown Toronto,0,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Bubble Tea Shop,Theater
2,Downtown Toronto,0,Coffee Shop,Cosmetics Shop,Hotel,Italian Restaurant,Cocktail Bar,Clothing Store,Café,Park,Seafood Restaurant,Gym
3,East Toronto,0,Health Food Store,Pub,Trail,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
4,Downtown Toronto,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Breakfast Spot,Farmers Market,Pharmacy,Restaurant,Cheese Shop,Beer Bar
5,Downtown Toronto,0,Coffee Shop,Clothing Store,Plaza,Restaurant,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Sandwich Place,Café,Bookstore
6,Downtown Toronto,0,Café,Grocery Store,Italian Restaurant,Playground,Baby Store,Athletics & Sports,Coffee Shop,Candy Store,Farmers Market,Farm
7,Downtown Toronto,0,Coffee Shop,Hotel,Café,Restaurant,Gym,Steakhouse,American Restaurant,Salad Place,Asian Restaurant,Japanese Restaurant
8,West Toronto,0,Park,Grocery Store,Bakery,Bus Line,Liquor Store,Middle Eastern Restaurant,Brazilian Restaurant,Furniture / Home Store,Café,Bar
10,Downtown Toronto,0,Coffee Shop,Hotel,Japanese Restaurant,Deli / Bodega,Aquarium,Park,Boat or Ferry,Plaza,Electronics Store,Sporting Goods Shop


In [100]:
torro_merged.loc[torro_merged['Cluster Labels'] == 1, torro_merged.columns[[1] + list(range(5, torro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East Toronto,1,Park,Intersection,Yoga Studio,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
22,West Toronto,1,Convenience Store,Park,Residential Building (Apartment / Condo),Yoga Studio,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
23,Central Toronto,1,Gym Pool,Playground,Park,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
29,Central Toronto,1,Convenience Store,Gym,Playground,Park,Yoga Studio,Dry Cleaner,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
33,Downtown Toronto,1,Park,Playground,Bike Trail,Yoga Studio,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [101]:
torro_merged.loc[torro_merged['Cluster Labels'] == 2, torro_merged.columns[[1] + list(range(5, torro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Health & Beauty Service,Costume Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [102]:
torro_merged.loc[torro_merged['Cluster Labels'] == 3, torro_merged.columns[[1] + list(range(5, torro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,3,Park,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


In [103]:
torro_merged.loc[torro_merged['Cluster Labels'] == 4, torro_merged.columns[[1] + list(range(5, torro_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,East Toronto,4,Bus Line,Business Service,Grocery Store,Park,Ice Cream Shop,Discount Store,Yoga Studio,Eastern European Restaurant,Fast Food Restaurant,Farmers Market
18,Central Toronto,4,Bus Line,Swim School,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
